# SFT fine-tuning with QWen2.5 for Alpaca Dataset
(https://github.com/unslothai/notebooks/blob/main/nb/Qwen2.5_(7B)-Alpaca.ipynb)

### Installation

In [1]:
%%capture

# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

In [2]:
import torch
print(torch.__version__)

2.6.0+cu124


In [3]:
import bitsandbytes
print(bitsandbytes.__version__)

0.45.5


In [4]:
import transformers
print(transformers.__version__)

4.51.3


# Load Model and Tokenizer

In [12]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [14]:
from peft import LoraConfig, get_peft_model

In [14]:
model_id = "Qwen/Qwen2.5-0.5B"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Quantization config (QLoRA)


In [12]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

### *LORA* Config

In [15]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ]
)

## DATA

We now use the Alpaca dataset from yahma, which is a filtered version of 52K of the original Alpaca dataset. You can replace this code section with your own data prep.

[NOTE] To train only on completions (ignoring the user's input) read TRL's docs here.

[NOTE] Remember to add the EOS_TOKEN to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the llama-3 template for ShareGPT datasets, try our conversational notebook-Alpaca.ipynb)

For text completions like novel writing, try this notebook-Text_Completion.ipynb).


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""





In [16]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")


In [17]:
len(dataset)

51760

In [18]:
dataset[0]

{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'input': '',
 'instruction': 'Give three tips for staying healthy.'}

In [19]:
dataset = dataset.map(formatting_prompts_func, batched = True)


Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [21]:
dataset[0]

{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes th

In [22]:
dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 51760
})

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [23]:
from trl import SFTTrainer, SFTConfig


In [25]:
# Define training config first
training_args = SFTConfig(
    dataset_text_field="text",           # The field in your dataset containing text prompts
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,       # Accumulate gradients to mimic larger batch size
    warmup_steps=5,
    # num_train_epochs = 1, # Set this for 1 full training run.
    max_steps=30,                        # Train for 30 steps (for quick tests)
    learning_rate=2e-4,                  # Learning rate, adjust for longer training
    logging_steps=1,
    optim="adamw_8bit",                  # Optimizer with 8-bit AdamW (efficient on smaller GPUs)
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    report_to="none"                    # Disable reporting (like WandB)
)


In [28]:
# Instantiate the trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
)

<ipython-input-28-b94c391223a5>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/51760 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/51760 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [29]:
# Optionally print trainable params info
trainer.model.print_trainable_parameters()


trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497


In [30]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.156 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [31]:
trainer_stats = trainer.train()

Step,Training Loss
1,1.728200
2,2.057200
3,1.826900
4,2.108300
5,1.810400
6,1.732300
7,1.222300
8,1.525000
9,1.395100
10,1.424000


In [32]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

62.2084 seconds used for training.
1.04 minutes used for training.
Peak reserved memory = 10.289 GB.
Peak reserved memory for training = 9.133 GB.
Peak reserved memory % of max memory = 69.799 %.
Peak reserved memory for training % of max memory = 61.956 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [40]:
input = tokenizer(
    [
        alpaca_prompt.format(
            "Continue the fibonnaci sequence.", # instruction
            "1, 1, 2, 3, 5, 8", # input
            "", # output - leave this blank for generation!
        )
    ],
    return_tensors="pt",
).to("cuda")

In [41]:
input

{'input_ids': tensor([[38214,   374,   458,  7600,   429, 16555,   264,  3383,    11, 34426,
           448,   458,  1946,   429,  5707,  4623,  2266,    13,  9645,   264,
          2033,   429, 34901, 44595,   279,  1681,   382, 14374, 29051,   510,
         23526,   279, 15801, 26378, 24464,  8500,   382, 14374,  5571,   510,
            16,    11,   220,    16,    11,   220,    17,    11,   220,    18,
            11,   220,    20,    11,   220,    23,   271, 14374,  5949,   510]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [42]:
outputs = model.generate(**input, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1']

In [44]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6705, 10946, 17711, 28657, 46368, 75025, 121393


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [45]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set

---

`False` to `True`:

In [50]:
if False:
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "trainer_output/checkpoint-30/", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [51]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 243, 443, 748, 1234, 2177, 3943, 6368, 10843, 17374, 28125, 45489, 73744, 121529, 197


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

## merge with lora and support vllm

In [56]:
from peft import AutoPeftModelForCausalLM


In [57]:

base_model_id = "Qwen/Qwen2.5-0.5B"     # Base HF model
adapter_path = "trainer_output/checkpoint-30/"   # Your fine-tuned LoRA folder
output_path = "merged_model"            # Where to save merged model

# Load PEFT model with LoRA adapter
model = AutoPeftModelForCausalLM.from_pretrained(adapter_path, device_map="auto", torch_dtype="auto")

# Merge LoRA weights into base model weights
model = model.merge_and_unload()

# Save merged model for vLLM usage
model.save_pretrained(output_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.save_pretrained(output_path)


('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/vocab.json',
 'merged_model/merges.txt',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

## Generate Text via vLLM

In [59]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [8]:
from vllm import LLM, SamplingParams

In [3]:
max_model_len = 2048
merged_model_path = "merged_model"            # Where to save merged model

# Load vLLM model
v_model = LLM(
    model=merged_model_path,
    enforce_eager=True,            # Optional, for debugging/tracing
    dtype="half",
    max_model_len=max_model_len,
    trust_remote_code=True,
)


WARNING 05-20 00:11:43 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-20 00:11:43 [config.py:717] This model supports multiple tasks: {'generate', 'embed', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
WARNING 05-20 00:11:43 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 05-20 00:11:43 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='merged_model', speculative_config=None, tokenizer='merged_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guid

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-20 00:11:51 [loader.py:458] Loading weights took 5.00 seconds
INFO 05-20 00:11:51 [model_runner.py:1140] Model loading took 0.9277 GiB and 5.429539 seconds
INFO 05-20 00:11:52 [worker.py:287] Memory profiling takes 1.11 seconds
INFO 05-20 00:11:52 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 05-20 00:11:52 [worker.py:287] model weights take 0.93GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 10.90GiB.
INFO 05-20 00:11:53 [executor_base.py:112] # cuda blocks: 59547, # CPU blocks: 21845
INFO 05-20 00:11:53 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 465.21x
INFO 05-20 00:11:57 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 6.50 seconds


In [7]:
prompt_text = alpaca_prompt.format(
    "Continue the fibonnaci sequence.",  # instruction
    "1, 1, 2, 3, 5, 8",                 # input
    "",                                 # output (empty for generation)
)


In [9]:
SAMPLING_PARAMS = {
    "temperature": 0.0,
    "top_p": 0.98,
    "max_tokens": 256,
}

In [10]:
# Instead of tokenized input, just pass prompt_text as a string:
outputs = v_model.generate(prompt_text, SamplingParams(**SAMPLING_PARAMS))

print(outputs[0].outputs[0].text)


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155
